In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torchvision
import torchvision.models as models
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
model_resnet18 = torch.hub.load('pytorch/vision', 'resnet18', pretrained = True)
model_resnet34 = torch.hub.load('pytorch/vision', 'resnet34', pretrained = True)

Using cache found in /home/ubuntu/.cache/torch/hub/pytorch_vision_master
Using cache found in /home/ubuntu/.cache/torch/hub/pytorch_vision_master


In [3]:
for name, param in model_resnet18.named_parameters():
    if("bn" not in name):
        param.requires_grad = False

for name, param in model_resnet34.named_parameters():
    if("bn" not in name):
        param.requires_grad = False

In [4]:
num_classes = 2

model_resnet18.fc = nn.Sequential(nn.Linear(model_resnet18.fc.in_features, 512),
                                 nn.ReLU(),
                                 nn.Dropout(),
                                 nn.Linear(512, num_classes))
model_resnet34.fc = nn.Sequential(nn.Linear(model_resnet34.fc.in_features, 512),
                                 nn.ReLU(),
                                 nn.Dropout(),
                                 nn.Linear(512, num_classes))

In [5]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=5, device="cpu"):
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
                        
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets).view(-1)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.4f}, Validation Loss: {:.4f}, accuracy = {:.4f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

In [6]:
import os
original_path = "./data/cats-vs-dogs"
subdirs = ['train', 'validation', 'test']
for subdir in subdirs:
    newdir = os.path.join(original_path, subdir)
    for subset in ['cats', 'dogs']:
        newsubset = os.path.join(newdir, subset)
        os.makedirs(newsubset, exist_ok =True)
    
original_train_path = "./data/train"
train_path = "./data/cats-vs-dogs/train"
validation_path = "./data/cats-vs-dogs/validation"
test_path = "./data/cats-vs-dogs/test"

import re
import shutil
for filename in os.listdir(original_train_path):
    index = re.findall(r"\d+", filename)[0]
    category = filename.split('.')[0]
    src = os.path.join(original_train_path, filename)
    newfilename=index+".jpg"
    if int(index) > 10249:
        shutil.move(src, os.path.join(validation_path, category+'s', newfilename))
    elif int(index) > 7999:
        shutil.move(src, os.path.join(test_path, category+'s', newfilename))
    else:
        shutil.move(src, os.path.join(train_path, category+'s', newfilename))

In [7]:
batch_size = 32
img_dimensions =224

img_transforms = transforms.Compose([
    transforms.Resize((img_dimensions, img_dimensions)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

img_test_transforms = transforms.Compose([
    transforms.Resize((img_dimensions, img_dimensions)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_data_path = train_path
train_data = torchvision.datasets.ImageFolder(root=train_data_path, transform=img_transforms)
validation_data_path = validation_path
validation_data = torchvision.datasets.ImageFolder(root=validation_data_path, 
                                                  transform=img_test_transforms)

test_data_path = test_path
test_data = torchvision.datasets.ImageFolder(root=test_data_path, transform=img_test_transforms)

num_workers = 6
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                                shuffle=True, num_workers = num_workers)
validation_data_loader = torch.utils.data.DataLoader(validation_data, batch_size=batch_size,
                                               shuffle=False, num_workers = num_workers)
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,
                                         shuffle=False, num_workers = num_workers)

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [8]:
print(f'Num training images: {len(train_data_loader.dataset)}')
print(f'Num validation images: {len(validation_data_loader.dataset)}')
print(f'Num test images: {len(test_data_loader.dataset)}')

Num training images: 16000
Num validation images: 4500
Num test images: 4500


In [9]:
def test_model(model):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_data_loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('correct: {:d} total: {:d}'.format(correct, total))
    print('accuracy: {:f}'.format(correct/total))

In [10]:
model_resnet18.to(device)
optimizer = optim.Adam(model_resnet18.parameters(), lr=0.001)
train(model_resnet18, optimizer, torch.nn.CrossEntropyLoss(), 
     train_data_loader, validation_data_loader, epochs = 100, device = device)

Epoch: 0, Training Loss: 0.0897, Validation Loss: 0.0635, accuracy = 0.9744
Epoch: 1, Training Loss: 0.0454, Validation Loss: 0.0344, accuracy = 0.9884
Epoch: 2, Training Loss: 0.0380, Validation Loss: 0.0335, accuracy = 0.9884
Epoch: 3, Training Loss: 0.0363, Validation Loss: 0.0393, accuracy = 0.9844
Epoch: 4, Training Loss: 0.0318, Validation Loss: 0.0407, accuracy = 0.9862
Epoch: 5, Training Loss: 0.0269, Validation Loss: 0.0434, accuracy = 0.9862
Epoch: 6, Training Loss: 0.0228, Validation Loss: 0.0667, accuracy = 0.9798
Epoch: 7, Training Loss: 0.0230, Validation Loss: 0.0428, accuracy = 0.9867
Epoch: 8, Training Loss: 0.0211, Validation Loss: 0.0504, accuracy = 0.9840
Epoch: 9, Training Loss: 0.0188, Validation Loss: 0.0540, accuracy = 0.9853
Epoch: 10, Training Loss: 0.0163, Validation Loss: 0.0514, accuracy = 0.9871
Epoch: 11, Training Loss: 0.0180, Validation Loss: 0.0533, accuracy = 0.9849
Epoch: 12, Training Loss: 0.0176, Validation Loss: 0.0730, accuracy = 0.9804
Epoch: 13

In [11]:
test_model(model_resnet18)

correct: 4427 total: 4500
accuracy: 0.983778


In [12]:
model_resnet34.to(device)
optimizer = optim.Adam(model_resnet34.parameters(), lr=0.001)
train(model_resnet34, optimizer, torch.nn.CrossEntropyLoss(),
      train_data_loader, validation_data_loader, epochs = 100, device =device)

Epoch: 0, Training Loss: 0.0686, Validation Loss: 0.0303, accuracy = 0.9884
Epoch: 1, Training Loss: 0.0346, Validation Loss: 0.0345, accuracy = 0.9869
Epoch: 2, Training Loss: 0.0253, Validation Loss: 0.0364, accuracy = 0.9869
Epoch: 3, Training Loss: 0.0209, Validation Loss: 0.0343, accuracy = 0.9862
Epoch: 4, Training Loss: 0.0220, Validation Loss: 0.0424, accuracy = 0.9880
Epoch: 5, Training Loss: 0.0185, Validation Loss: 0.0401, accuracy = 0.9849
Epoch: 6, Training Loss: 0.0201, Validation Loss: 0.0382, accuracy = 0.9884
Epoch: 7, Training Loss: 0.0136, Validation Loss: 0.0471, accuracy = 0.9871
Epoch: 8, Training Loss: 0.0171, Validation Loss: 0.0447, accuracy = 0.9871
Epoch: 9, Training Loss: 0.0131, Validation Loss: 0.0449, accuracy = 0.9871
Epoch: 10, Training Loss: 0.0111, Validation Loss: 0.0461, accuracy = 0.9844
Epoch: 11, Training Loss: 0.0151, Validation Loss: 0.0833, accuracy = 0.9722
Epoch: 12, Training Loss: 0.0141, Validation Loss: 0.0586, accuracy = 0.9864
Epoch: 13

In [13]:
test_model(model_resnet34)

correct: 4433 total: 4500
accuracy: 0.985111


In [14]:
def find_classes(dir):
    classes = os.listdir(dir)
    classes.sort()
    class_to_idx = {classes[i]: i for i in range(len(classes))}
    return classes, class_to_idx

def make_prediction(model, filename):
    labels, _  = find_classes(test_data_path)
    img = Image.open(filename)
    img = img_test_transforms(img)
    img = img.unsqueeze(0)
    prediction = model(img.to(device))
    prediction = prediction.argmax()
    print(labels[prediction])

In [15]:
make_prediction(model_resnet34, os.path.join(test_data_path, "dogs", "10245.jpg"))
make_prediction(model_resnet34, os.path.join(test_data_path, "cats", "10245.jpg"))

dogs
cats


In [16]:
torch.save(model_resnet18.state_dict(), "./model_resnet18.pth")
torch.save(model_resnet34.state_dict(), "./model_resnet34.pth")

resnet18 = torch.hub.load('pytorch/vision', 'resnet18')
resnet18.fc = nn.Sequential(nn.Linear(resnet18.fc.in_features, 512),
                           nn.ReLU(), nn.Dropout(), 
                           nn.Linear(512, num_classes))
resnet18.load_state_dict(torch.load('./model_resnet18.pth'))
resnet18.eval()

resnet34 = torch.hub.load('pytorch/vision', 'resnet34')
resnet34.fc = nn.Sequential(nn.Linear(resnet34.fc.in_features, 512),
                           nn.ReLU(), nn.Dropout(),
                           nn.Linear(512, num_classes))
resnet34.load_state_dict(torch.load('./model_resnet34.pth'))
resnet34.eval()

Using cache found in /home/ubuntu/.cache/torch/hub/pytorch_vision_master
Using cache found in /home/ubuntu/.cache/torch/hub/pytorch_vision_master


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [17]:
models_ensemble = [resnet18.to(device), resnet34.to(device)]
correct = 0
total = 0
with torch.no_grad():
    for data in test_data_loader:
        images, labels = data[0].to(device), data[1].to(device)
        predictions = [i(images).data for i in models_ensemble]
        avg_predictions = torch.mean(torch.stack(predictions), dim=0)
        _, predicted = torch.max(avg_predictions, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
print('accuracy = {:f}'.format(correct/total))
print('correct: {:d} total: {:d}'.format(correct, total))

accuracy = 0.988667
correct: 4449 total: 4500
